# 37009 Workshop Week 6: Analytical VaR

## Background

Today, 21 December 2021, we hold a portfolio consisting of bonds and physical assets. The specification of each component is as follows:
1. A coupon bond with face value of \$2 million paying semi-annual coupons at a rate of 4\% per annum which matures on 21 December 2022.
2. A coupon bond with face value of \$2 million paying semi-annual coupons at a rate of 2\% per annum which matures on 21 December 2023.
3. A short position on 1000 shares of the ASX 200
4. A long position on 2500 shares of S\&P 500 (*Note: Prices provided in the data set are prices of the S\&P500 in USD*)
5. A long position on US\$4 million

## Tasks

Given the time series of financial data:

1. Determine the marked-to-market value of the portfolio today.
2. Determine the risk factors driving the portfolio. Calculate the daily continuously compounded returns (log-returns) of the risk factors using the data provided.
3. Compute the **undiversified** one-day and 10-day 90\% value-at-risk for the entire portfolio using the delta-normal approach (*Hint: You will need to determine the VaR for each portfolio component/instrument*)
4. Compute the **diversified** one-day and 10-day 90\% value-at-risk for the entire portfolio using the delta-normal approach (*Hint: You will need to compute the variance-covariance matrix of the returns of the risk factors*)

**NOTE:** We will use the log-returns of the risk factors to calculate the VaR, since using log-returns allow us to perform square-root-of-time scaling to compute the 10-day VaR from the 1-day VaR. The time scaling can be used on the VaR computed from percentage returns, although this adds another layer of approximation.

## Utilities

This section loads the required libraries and the cash flow mapping function which may be useful in mapping the cash flows of the bonds in the portfolio. We also load the historical data we are provided.

**NOTE:** For simplicity, zero rate, index, and exchange rate data have been provided for the same trading days. If you have acquired data from different sources, this may not be the case. As such, you will need to merge multiple data sets using some joining rule; for example, keeping the data for the dates that are common to your data sources.

In [2]:
# Required libraries
import numpy as np
import scipy as sp
import scipy.stats
import pandas as pd
import math
from datetime import datetime
from datetime import timedelta

# Cash flow mapping function
def CFmap2V(maturity, left_maturity, right_maturity, map_type = "Duration", 
            left_vol = None, right_vol = None, left_right_cor = None):
    
    # Assumption: Cash flow at non-standard maturity is mapped to two vertices, one on the left and one on the right.
    # Imposes PV-invariance; may impose duration-invariance or volatility-invariance, but not both.
    
    # Output: alpha parameter to allocate a PV of 1 to the left vertex T1. 
    # For the volatility mapping, the function returns two values of alpha. 
    # We often will choose the alpha that is between 0 and 1.
    
    # Use mathematical notation for function inputs
    T = maturity
    T1 = left_maturity
    T2 = right_maturity
    
    if map_type == "Duration":
        alpha = (T2 - T) / (T2 - T1)
        return alpha
    
    if map_type == "Volatility":
        
        if (left_vol == None or right_vol == None or left_right_cor == None):
            print("Please provide non-empty volatility-invariant mapping inputs")
        
        else:
            # Extract volatility-invariant CF map inputs
            sigma1 = left_vol
            sigma2 = right_vol
            rho = left_right_cor
            
            # Linearly interpolate the variance at the non-standard maturity
            sq_sigma_interp = np.interp(x = T, xp = np.array([T1, T2]), fp = np.array([sigma1 ** 2, sigma2 ** 2]))
            
            # Coefficients of quadratic equation for alpha
            a_coeff = sigma1 ** 2 - 2 * rho * sigma1 * sigma2 + sigma2 ** 2
            b_coeff = 2 * rho * sigma1 * sigma2 - 2 * sigma2 ** 2
            c_coeff = sigma2 ** 2 - sq_sigma_interp
            
            # Solve quadratic equation for alpha
            alpha1 = (-b_coeff + np.sqrt(b_coeff ** 2 - 4 * a_coeff * c_coeff)) / (2 * a_coeff)
            alpha2 = (-b_coeff - np.sqrt(b_coeff ** 2 - 4 * a_coeff * c_coeff)) / (2 * a_coeff)
            alpha = np.array([alpha1, alpha2])
            
            return alpha

## Data Pre-processing

In [3]:
# Load historical data
fin_data = pd.read_csv('C:/Users/leeha/WorkshopWeek4Data-1.csv', index_col = 0)

# Write the exchange rates as XX AUD/USD
fin_data['FXrate'] = 1 / fin_data['AUDUSD']

# # Compute historical prices of S&P500 in AUD
# fin_data['SPXCompAUD'] = fin_data['SPXComp'] * fin_data['FXrate']a
# fin_data.tail()

# Compute zero coupon bond prices
maturities = np.array([1/360, 1/12, 2/12, 3/12, 6/12, 9/12, 1, 2, 3, 4, 5, 6, 7, 8, 9])
zcb = np.exp(-fin_data.iloc[:, 3:(3 + len(maturities))] * maturities / 100)

# Extract data for mark-to-market valuation (observations on 21 December 2021)
fin_data_today = fin_data.loc['2021-12-21']
zcb_today = zcb.loc['2021-12-21']

# Compute log-returns for all market variables
fin_data_logret = np.log(fin_data)
fin_data_logret = fin_data_logret.diff().tail(-1)
zcb_logret = np.log(zcb)
zcb_logret = zcb_logret.diff().tail(-1)

In [3]:
# fin_data_logret.head()

In [4]:
# zcb_logret.head()

## Mark-to-Market Valuation

### Physical Assets

The valuation of the phyiscal asset holdings is straightforward; simply multiply the number of units by the spot price today. It will also be helpful to express the valuation mathematically to facilitate the risk factor mapping for the entire portfolio. We have

$$\begin{align*}
V_{\text{ASX}}(t) & = 1,000 \cdot S_{\text{ASX}}(t)\\
V_{\text{SPX}}(t) & = 2,500 \cdot S_{\text{SPX}}(t) \cdot S_{\text{FX}}(t)\\
V_{\text{USD}}(t) & = 4,000,000 \cdot S_{\text{FX}}(t)
\end{align*}$$

where the $S_{\text{FX}}(t)$ is the spot exchange rate in AUD per USD and the other variables are indicated by the subscript.

**EXERCISE:** How will the analysis differ if we convert the price of the SPX to AUD and use the AUD price as a risk factor instead of both the USD price of the SPX and the exchange rate?

In [5]:
# Mark-to-market valuation for physical assets

# Number of units (negative for short position)
N_ASX = -1000
N_SPX = 2500
N_USD = 4000000

# Valuation
V_ASX = N_ASX * fin_data_today['ASX200']
V_SPX = N_SPX * fin_data_today['SPXComp'] * fin_data_today['FXrate']
V_USD = N_USD * fin_data_today['FXrate']

print(V_ASX, V_SPX, V_USD)

-6060358.0 8713873.019994806 5193456.245131135


### Bonds

The table below summarizes the cash flows from the bonds:

| Maturity | 6 months | 1 year | 1.5 years | 2 years |
|:-|---|---|---|---|
| Bond 1 | 40,000 | 2,040,000 | | |
| Bond 2 | 20,000 | 20,000 | 20,000 | 2,020,000|

We do not have data on the zero rates with a 1.5-year maturity. There are two ways to go about this:

1. For valuation purposes, interpolate the 1.5-year zero rate from the 1-year and 2-year zero rates
2. Perform a present-value invariant cash flow mapping to map the 1.5-year CF to the 1-year and 2-year standard vertices.

As we will eventually need to map the 1.5-year cash flow of Bond 2 anyway for risk management purposes, we can perform this step now in the valuation as well. Since PV-invariance is imposed, we will get the same PV and, hence, the same marked-to-market value of the bond. 

#### Cash Flow Mapping

**A PV- and volatility-invariant cash flow map will be used.** To this end, we need to estimate the standard deviations and correlation of the log-returns of zero coupon bonds with 1-year and 2-year maturities.

In [6]:
# Compute standard deviation and correlation of log-returns of the 1-year and 2-year zero coupon bonds
zcb_1y_std = np.std(zcb_logret['AU01Y00'])
zcb_2y_std = np.std(zcb_logret['AU02Y00'])
zcb_1y2y_corr = np.corrcoef(zcb_logret['AU01Y00'], zcb_logret['AU02Y00'])[1,0]

# Compute cash flow mapping coefficient
cf_map_coef = CFmap2V(1.5, 1, 2, "Volatility", zcb_1y_std, zcb_2y_std, zcb_1y2y_corr)
cf_map_coef = cf_map_coef[np.where((cf_map_coef >= 0) & (cf_map_coef <= 1))[0]].item()     # .item() so that it returns a float rather than a singleton array

print(cf_map_coef, 20000 * cf_map_coef, 20000 * (1 - cf_map_coef))

0.34368856595819197 6873.7713191638395 13126.22868083616


The mapped cash flows are as follows (manually entered):

| Maturity | 6 months | 1 year | 1.5 years | 2 years |
|:-|---|---|---|---|
| Bond 1 | 40,000.00 | 2,040,000.00 | | |
| Bond 2 | 20,000.00 | 20,000.00 | 20,000.00 | 2,020,000.00
| Bond 2 (Mapped) | 20,000.00 | 26,873.77 | | 2,033,126.23|

#### Mark-to-Marked Valuation

We are now ready to compute the marked-to-market values of the two bonds, which are given by

$$\begin{align*}
V_{B_1}(t) & = 40,000 \cdot P(0,0.5) + 2,040,000 \cdot P(0,1)\\
V_{B_2}(t) & = 20,000 \cdot P(0,0.5) + 26,873.77 \cdot P(0,1) + 2,033,126.23 \cdot P(0,2)
\end{align*}$$

In [7]:
# Bond parameters
B1_fv = 2000000
B1_coup_rate = 0.04
B1_coup = B1_fv * B1_coup_rate / 2

B2_fv = 2000000
B2_coup_rate = 0.02
B2_coup = B2_fv * B2_coup_rate / 2

# Valuation (manually done since there are only a few CFs)
V_B1 = B1_coup * zcb_today['AU00Y06'] + (B1_fv + B1_coup) * zcb_today['AU01Y00']
V_B2 = B2_coup * zcb_today['AU00Y06'] + (B2_coup + cf_map_coef * B2_coup) * zcb_today['AU01Y00'] \
        + (B2_fv + B2_coup + (1 - cf_map_coef) * B2_coup) * zcb_today['AU02Y00']

print(V_B1, V_B2)

2039883.2875202913 1998600.8209526178


The total value of the portfolio is the sum of the all the marked-to-market values computed above.

In [8]:
V_portfolio = V_ASX + V_SPX + V_USD + V_B1 + V_B2
V_portfolio

11885455.373598851

## Risk Factor Mapping

From the mark-to-market valuation step, we conclude that the risk factors for the portfolio are the ASX spot price, the S\&P 500 spot price (in USD), the spot exchange rate in AUD per USD, and the prices of zero coupon bonds with a 6-month, a 1-year, and a 2-year maturity.

With respect to these risk factors, a delta approximation for the change in the value of each component of the portfolio is given by

$$\begin{align*}
\Delta V_{\text{ASX}} & = N_{\text{ASX}} \cdot \Delta S_{\text{ASX}} \\
\Delta V_{\text{SPX}} & = N_{\text{SPX}} \cdot S_{\text{FX}} \cdot \Delta S_{\text{SPX}} + N_{\text{SPX}} \cdot S_{\text{SPX}} \cdot \Delta S_{\text{FX}} \\
\Delta V_{\text{USD}} & = N_{\text{USD}} \cdot \Delta S_{\text{FX}} \\
\Delta V_{B_1} & = C_{0.5}^{(1)} \cdot \Delta P_{0.5} + C_1^{(1)} \cdot \Delta P_1\\
\Delta V_{B_2} & = C_{0.5}^{(2)} \cdot \Delta P_{0.5} + C_1^{(2)} \cdot \Delta P_1 + C_2^{(2)} \cdot \Delta P_2,
\end{align*}$$

where $C_t^{(i)}$ is the cash flow (coupon payment and/or principal, including mapped CFs) for bond $i$ at time $t$, for $i=1,2$ and $t = 0.5,1,2$, and $P_t$ is the price of a zero coupon bond with maturity $t$. Above, we omit the time argument '$(t)$.'

Using the approximation $\Delta \ln x \approx \frac{\Delta x}{x}$, we can construct a mapping with respect to log-returns as follows:

$$\begin{align*}
\Delta V_{\text{ASX}} & = N_{\text{ASX}} \cdot S_{\text{ASX}}\cdot \Delta \ln S_{\text{ASX}} \\
                      & = V_{\text{ASX}} \cdot \Delta \ln S_{\text{ASX}} \\
\Delta V_{\text{SPX}} & = N_{\text{SPX}} \cdot S_{\text{FX}} \cdot S_{\text{SPX}} \cdot \Delta \ln S_{\text{SPX}} + N_{\text{SPX}} \cdot S_{\text{FX}} \cdot S_{\text{SPX}} \cdot \Delta \ln S_{\text{FX}} \\
                      & = V_{\text{SPX}} \cdot \Delta \ln S_{\text{SPX}} + V_{\text{SPX}} \cdot \Delta \ln S_{\text{FX}} \\
\Delta V_{\text{USD}} & = N_{\text{USD}} \cdot S_{\text{FX}} \cdot \Delta \ln S_{\text{FX}} \\
                      & = V_{\text{USD}} \cdot \Delta \ln S_{\text{FX}} \\
\Delta V_{B_1} & = C_{0.5}^{(1)} \cdot P_{0.5} \cdot \Delta \ln P_{0.5} + C_1^{(1)} \cdot P_1 \cdot \Delta \ln P_1\\
\Delta V_{B_2} & = C_{0.5}^{(2)} \cdot P_{0.5} \cdot \Delta \ln P_{0.5} + C_1^{(2)} \cdot P_1 \cdot \Delta \ln P_1 + C_2^{(2)} \cdot P_2 \cdot \Delta \ln P_2.
\end{align*}$$

(Unfortunately, the change in bond value cannot be simplified in terms of the current value of the bond.)

### Covariance matrix of risk factors

For convenience, we collect in a single data frame the log-returns of the relevant risk factors. We then calculate the covariance matrix from this data frame.

In [9]:
# Collect the log-returns of all relevant risk factors in a single data frame
RF_logret = pd.DataFrame({'ASX200' : fin_data_logret['ASX200'], 'SPXComp' : fin_data_logret['SPXComp'],
                         'FXrate' : fin_data_logret['FXrate'], 'AU00Y06' : zcb_logret['AU00Y06'],
                         'AU01Y00' : zcb_logret['AU01Y00'], 'AU02Y00' : zcb_logret['AU02Y00']})

# Compute the covariance matrix
RF_logret_cov = RF_logret.cov()

RF_logret_cov

,ASX200,SPXComp,FXrate,AU00Y06,AU01Y00,AU02Y00
ASX200,7.531915e-05,1.290643e-05,-6.772445e-06,-3.468750e-08,-1.127068e-07,-1.206142e-07
SPXComp,1.290643e-05,5.885604e-05,-1.631323e-05,2.823914e-09,-1.620656e-07,-9.802465e-07
FXrate,-6.772445e-06,-1.631323e-05,4.259371e-05,1.105966e-07,4.137853e-07,1.238799e-06
AU00Y06,-3.468750e-08,2.823914e-09,1.105966e-07,6.544504e-09,1.168089e-08,1.823196e-08
AU01Y00,-1.127068e-07,-1.620656e-07,4.137853e-07,1.168089e-08,3.423297e-08,9.094777e-08
AU02Y00,-1.206142e-07,-9.802465e-07,1.238799e-06,1.823196e-08,9.094777e-08,4.036768e-07


In [10]:
RF_logret_cov.loc[['ASX200', 'SPXComp'], ['ASX200', 'SPXComp']]

,ASX200,SPXComp
ASX200,0.000075,0.000013
SPXComp,0.000013,0.000059


## Undiversified VaR

To compute the undiversified VaR of the portfolio, we must compute the VaR of each component based on the assumed probability distribution of its respective risk factors. For each instrument/component, we assume that the log-returns of its risk factors $\mathbf{R}^\ell_x$ has a (multivariate) normal distribution with mean $\mathbb{E}(\mathbf{R}_x^\ell) = \mathbf{0}$ and covariance matrix $\mathbf{\Sigma} := \widehat{\text{Cov}(\mathbf{R}_x^\ell)}$, the covariance matrix estimated from the time series of one-day log-returns (**delta-normal approach**). 

Since the change in the value of each component can be written as $\Delta V = \mathbf{W}^\top \mathbf{R}_x^\ell$, where $\mathbf{W}$ is the vector of (dollar) exposures to each risk factor in $\mathbf{R}_x^\ell$, the one-day VaR with confidence level $c\in(0,1)$ obtained from the delta-normal approach is given by 

$$\text{VaR}_c = -\Phi^{-1}(1-c) \sqrt{\mathbf{W}^\top \mathbf{\Sigma} \mathbf{W}},$$

where $\Phi^{-1}(\cdot)$ is the inverse cdf of the standard normal distribution.

We perform this calculation for each instrument as follows.

In [11]:
# VaR parameter
conf_level = 0.90

### ASX 200

In [12]:
# Exposure to and (co)variance of risk factor(s)
ASX_exp = V_ASX 
ASX_rf_cov = RF_logret_cov.loc['ASX200', 'ASX200']

# One-day VaR
ASX_VaR = -sp.stats.norm.ppf(1 - conf_level) * np.sqrt((ASX_exp ** 2) * ASX_rf_cov)

### S\&P 500

In [13]:
# Exposure to and (co)variance of risk factor(s)
SPX_exp = np.array([V_SPX, V_SPX]) 
SPX_rf_cov = RF_logret_cov.loc[['SPXComp', 'FXrate'], ['SPXComp', 'FXrate']]

# One-day VaR
SPX_variance = np.matmul(SPX_exp.transpose(), np.matmul(SPX_rf_cov, SPX_exp))
SPX_VaR = -sp.stats.norm.ppf(1 - conf_level) * np.sqrt(SPX_variance)

### USD

In [14]:
# Exposure to and (co)variance of risk factor(s)
USD_exp = V_USD 
USD_rf_cov = RF_logret_cov.loc['FXrate', 'FXrate']

# One-day VaR
USD_VaR = -sp.stats.norm.ppf(1 - conf_level) * np.sqrt((USD_exp ** 2) * USD_rf_cov)

### Bond 1

In [15]:
# Exposure to and (co)variance of risk factor(s)
B1_exp = np.array([B1_coup * zcb_today['AU00Y06'], (B1_fv + B1_coup) * zcb_today['AU01Y00']]) 
B1_rf_cov = RF_logret_cov.loc[['AU00Y06', 'AU01Y00'], ['AU00Y06', 'AU01Y00']]

# One-day VaR
B1_variance = np.matmul(B1_exp.transpose(), np.matmul(B1_rf_cov, B1_exp))
B1_VaR = -sp.stats.norm.ppf(1 - conf_level) * np.sqrt(B1_variance)

### Bond 2

In [16]:
# Exposure to and (co)variance of risk factor(s)
B2_exp = np.array([B2_coup * zcb_today['AU00Y06'], (B2_coup + cf_map_coef * B2_coup) * zcb_today['AU01Y00'],
                  (B2_fv + B2_coup + (1 - cf_map_coef) * B2_coup) * zcb_today['AU02Y00']]) 
B2_rf_cov = RF_logret_cov.loc[['AU00Y06', 'AU01Y00', 'AU02Y00'], ['AU00Y06', 'AU01Y00', 'AU02Y00']]

# One-day VaR
B2_variance = np.matmul(B2_exp.transpose(), np.matmul(B2_rf_cov, B2_exp))
B2_VaR = -sp.stats.norm.ppf(1 - conf_level) * np.sqrt(B2_variance)

### One-Day VaR

The undiversified one-day 90\% VaR for the portfolio is then the sum of one-day 90\% VaR of each instrument.

In [17]:
VaR_undiversified = ASX_VaR + SPX_VaR + USD_VaR + B1_VaR + B2_VaR
VaR_undiversified

205558.12661647573

**That is, over the next trading day, there is a 10\% probability that the loss in portfolio value will exceed AU\$ 205,558.13.**

### 10-Day VaR

The use of log-returns in the risk factor mapping allows us to use square-root-of-time scaling to compute the 10-day VaR. Specifically, we have

$$\text{10-day VaR} = \sqrt{10} \cdot \text{one-day VaR}.$$

For this particular portfolio, the undiversified 10-day VaR is

In [18]:
VaR_undiversified_10day = np.sqrt(10) * VaR_undiversified
VaR_undiversified_10day

650031.8716653443

**Intuitively, over (at the end of) the next 10 trading days, there is a 10\% probability that the loss in portfolio value will exceed AU\$ 650,031.87.**

**NOTE:** To compute the 10-day VaR without the time scaling technique, we need to use the 10-day returns instead of the one-day returns in $\mathbf{R}_x^\ell$. Computing the VaR using the formula above will (naturally) result to the 10-day VaR for the portfolio (or each instrument).

## Diversified VaR

Computing the diversified VaR for the entire portfolio involves determining the exposure to each relevant risk factor *at the level of the portfolio* and using the full covariance matrix of the log-returns of the risk factors.

From the risk factor maps above, we have the following expression

$$\left[\begin{array}{c}
\Delta V_{\text{ASX}} \\ \Delta V_{\text{SPX}} \\ \Delta V_{\text{USD}} \\ \Delta V_{B_1} \\ \Delta V_{B_2} 
\end{array}\right] = 
\left[\begin{array}{cccccc}
V_{\text{ASX}} & 0 & 0 & 0 & 0 & 0 \\
0 & V_{\text{SPX}} & V_{\text{SPX}} & 0 & 0 & 0 \\
0 & 0 & V_{\text{USD}} & 0 & 0 & 0 \\
0 & 0 & 0 & C_{0.5}^{(1)} P_{0.5} & C_1^{(1)} P_1 & 0 \\
0 & 0 & 0 & C_{0.5}^{(2)} P_{0.5} & C_1^{(2)} P_1 & C_2^{(2)} P_2
\end{array}\right]
\left[\begin{array}{c}
\Delta \ln S_{\text{ASX}} \\ \Delta \ln S_{\text{SPX}} \\ \Delta \ln S_{\text{FX}} \\ \Delta \ln P_{0.5} \\ \Delta \ln P_1 \\ \Delta \ln P_2
\end{array}\right].$$

At the portfolio level, the change in portfolio value exposed to each of the relevant risk factors in the right-most vector above. The dollar exposure of the portfolio with respect to each risk factor is given by the **column sums of the matrix above.** Thus, we have

$$\Delta V = \mathbf{W}^\top \mathbf{R}_x^\ell,$$ 

where $\Delta V$ is the change in the portfolio value, $\mathbf{R}_x^\ell$ is the vector of log-returns of the risk factors, and

$$\mathbf{W} = \left[\begin{array}{c}
V_{\text{ASX}} \\ V_{\text{SPX}} \\ V_{\text{SPX}} + V_{\text{USD}} \\ C_{0.5}^{(1)} P_{0.5} + C_{0.5}^{(2)} P_{0.5} \\ C_1^{(1)} P_1 + C_1^{(2)} P_1 \\ C_2^{(2)} P_2
\end{array}\right].$$

The (diversified) portfolio VaR can then be calculated from the formula above, where $\mathbf{\Sigma}$ is the covariance matrix of *all* relevant risk factors. The calculation is implemented below.

In [19]:
# Portfolio exposures
portfolio_exp = np.array([V_ASX, V_SPX, V_SPX + V_USD, 
                         B1_coup * zcb_today['AU00Y06'] + B2_coup * zcb_today['AU00Y06'],
                         (B1_fv + B1_coup) * zcb_today['AU01Y00'] + (B2_coup + cf_map_coef * B2_coup) * zcb_today['AU01Y00'],
                         (B2_fv + B2_coup + (1 - cf_map_coef) * B2_coup) * zcb_today['AU02Y00']])

# Portfolio variance
portfolio_variance = np.matmul(portfolio_exp.transpose(), np.matmul(RF_logret_cov, portfolio_exp))

# Diversified one-day VaR
VaR_diversified = -sp.stats.norm.ppf(1 - conf_level) * np.sqrt(portfolio_variance)
VaR_diversified

136579.26278637137

We observe that the diversified VaR AU \\$136,579.26 is markedly lower than the undiversified VaR AU \\$205,558.13. This reflects any diversification benefits arising from the historical dynamics of the risk factors. That is, it is possible that adverse movements in one risk factor is offset by beneficial movements in another, as captured by the full covariance matrix of the risk factors.

The 10-day VaR can likewise be approximated using square-root-of-time scaling.

In [20]:
VaR_diversified_10day = np.sqrt(10) * VaR_diversified
VaR_diversified_10day

431901.5515516087

The result is also lower than the scaled undiversified VaR.

**NOTE:** The diversified 10-day VaR can also be computed without scaling by using the 10-day returns instead of one-day returns in $\mathbf{R}_x^\ell$.